# Qualification Validation Logic Test

This notebook tests the `ValidationEngine` using the rules defined in `app/config/rules.toml` and mock applicant data.

In [1]:
import sys
from pathlib import Path
import json

# Get the project root directory (which is the parent of the 'app' directory)
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
    
from app.logic.validator import ValidationEngine
from app.logic.models import ApplicantData

## 1. Initialize the Engine

In [3]:
rules_path = Path.cwd() / 'config' / 'rules.toml'
engine = ValidationEngine(rules_path)

✅ Validation engine initialized with 2 qualifications.


## 2. Define Test Cases

We will create several mock applicants to test different eligibility paths.

In [4]:
test_cases = {
    "perfect_variant_1": ApplicantData(
        education="upper_secondary",
        work_experience_years=3.5,
        matching_experience_years=2.5,
        has_prior_level_4=True,
        base_training_hours=40
    ),
    "fails_variant_1_on_experience": ApplicantData(
        education="upper_secondary",
        work_experience_years=2, # Fails this check (requires 3)
        matching_experience_years=2,
        has_prior_level_4=True,
        base_training_hours=30
    ),
    "perfect_variant_2": ApplicantData(
        education="upper_secondary",
        work_experience_years=5,
        matching_experience_years=2.5,
        has_prior_level_4=False, # Not required, so this is fine
        base_training_hours=30
    ),
    "perfect_variant_4_or_5": ApplicantData(
        education="higher_or_technical",
        work_experience_years=3,
        matching_experience_years=1.5,
        base_training_hours=50
    ),
    "fails_all_variants": ApplicantData(
        education="upper_secondary",
        work_experience_years=1,
        matching_experience_years=0.5,
        has_prior_level_4=False,
        base_training_hours=10
    )
}

## 3. Run Validations and Print Results

In [5]:
for name, applicant_data in test_cases.items():
    print(f"\n--- Running test case: '{name}' ---")
    result = engine.validate(applicant_data, "toojuht_tase_5")
    # Pretty print the JSON result
    print(json.dumps(result, indent=2))


--- Running test case: 'perfect_variant_1' ---
{
  "qualification_id": "toojuht_tase_5",
  "results": [
    {
      "package_id": "variant_1",
      "is_met": true,
      "details": {
        "education": {
          "met": true,
          "required": "upper_secondary",
          "provided": "upper_secondary"
        },
        "total_experience": {
          "met": true,
          "required": 3,
          "provided": 3.5
        },
        "matching_experience": {
          "met": true,
          "required": 2,
          "provided": 2.5
        },
        "base_training": {
          "met": true,
          "required": 30,
          "provided": 40
        },
        "prior_level_4": {
          "met": true,
          "required": true,
          "provided": true
        }
      }
    },
    {
      "package_id": "variant_2",
      "is_met": false,
      "details": {
        "education": {
          "met": true,
          "required": "upper_secondary",
          "provided": "upper_secon